Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Market orders do not return price. So fetch trades. https://github.com/ccxt/ccxt/issues/798
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [32]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, quantity, price)
            orders.append(order)
        elif (len(ctx.record.portfolio.positions) > 0 
              and ctx.record.balance.get(c.ETH)[BalanceType.FREE] > 0.0):
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Backtest

In [33]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-09T00:00:00
---------------------------------------
OHLCV
    O: 0.0760 | C: 0.0766 | V: 7900.8 | T: 2018-01-09T00:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0766 | Quantity: 0.0100
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T00:30:00
---------------------------------------
OHLCV
    O: 0.0766 | C: 0.0774 | V: 7715.3 | T: 2018-01-09T00:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9992 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9984670200000001, 'used': 0.0, 'total': 0.9984670200000001}
    ETH - {'free': 0.02, 'used': 0.0, 'total': 0.02}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.01, 'cost_price': 0.076649, 'lates

Placing Order {
    "id": "36939728d94046cb9a3d1b079a0d40ba",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.076649,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T06:03:21.136925",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "36939728d94046cb9a3d1b079a0d40ba",
    "exchange_id": "paper",
    "exchange_order_id": "220f306037d74ee18add1548142eba84",
    "price": 0.076649,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T06:03:21.136925",
    "opened_time": null,
    "filled_time": "2018-01-18T06:03:21.194938",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
[{
    "id": "36939728d94046cb9a3d1b079

---------------------------------------
Epoch 4 - Timestep: 2018-01-09T01:30:00
---------------------------------------
OHLCV
    O: 0.0771 | C: 0.0764 | V: 14292.4 | T: 2018-01-09T01:30:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0000151400000001, 'used': 0.0, 'total': 1.0000151400000001}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.07740599999999999}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 5 - Timestep: 2018-01-09T02:00:00
---------------------------------------
OHLCV
    O: 0.0765 | C: 0.0773 | V: 10995.4 | T: 2018-01-09T02:00:00
ORDERS
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0000151400000001, 'used': 0.0, 'total': 1.0000151400000001}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/

[]
[]
Placing Order {
    "id": "a6c707f8b4424894b2682f97ac23d26f",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.078041,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-18T06:03:21.496796",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "a6c707f8b4424894b2682f97ac23d26f",
    "exchange_id": "paper",
    "exchange_order_id": "39c0c9b5cc164cdabc578ac62c3ea3e7",
    "price": 0.078041,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-18T06:03:21.496796",
    "opened_time": null,
    "filled_time": "2018-01-18T06:03:21.582998",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
[{
    "id": "a6c707f8b4424894b26

---------------------------------------
Epoch 7 - Timestep: 2018-01-09T03:00:00
---------------------------------------
OHLCV
    O: 0.0780 | C: 0.0785 | V: 8968.9 | T: 2018-01-09T03:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0785 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9992 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9984543200000002, 'used': 0.0, 'total': 0.9984543200000002}
    ETH - {'free': 0.02, 'used': 0.0, 'total': 0.02}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.01, 'cost_price': 0.078041, 'latest_price': 0.078041}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 8 - Timestep: 2018-01-09T03:30:00
---------------------------------------
OHLCV
    O: 0.0785 | C: 0.0791 | V: 10039.6 | T: 2018-01-09T03:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9984 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9968847000000003, '

...

---------------------------------------
Epoch 90 - Timestep: 2018-01-10T20:30:00
---------------------------------------
OHLCV
    O: 0.0901 | C: 0.0895 | V: 6886.1 | T: 2018-01-10T20:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9952 Total Val: 1.0006 PnL: 0.0006 Returns: 0.0006
BALANCE
    BTC - {'free': 0.9903316400000005, 'used': 0.0, 'total': 0.9903316400000005}
    ETH - {'free': 0.11999999999999998, 'used': 0.0, 'total': 0.11999999999999998}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.060000000000000005, 'cost_price': 0.09115498208333334, 'latest_price': 0.09012}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 91 - Timestep: 2018-01-10T21:00:00
---------------------------------------
OHLCV
    O: 0.0895 | C: 0.0901 | V: 7076.6 | T: 2018-01-10T21:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0901 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9961 Total Val: 1.0005 PnL: 0.0005 Returns

Placing Order {
    "id": "ee2708329a5d4ef9b2c16e53348b7886",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T06:03:28.269260",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "ee2708329a5d4ef9b2c16e53348b7886",
    "exchange_id": "paper",
    "exchange_order_id": "f343c2864f0c460798e8145a6a1b49b5",
    "price": 0.089488,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T06:03:28.269260",
    "opened_time": null,
    "filled_time": "2018-01-18T06:03:28.337377",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
[{
    "id": "ee2708329a5d4ef9b2c16e5334

    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 93 - Timestep: 2018-01-10T22:00:00
---------------------------------------
OHLCV
    O: 0.0872 | C: 0.0865 | V: 11784.5 | T: 2018-01-10T22:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9960 Total Val: 1.0004 PnL: 0.0004 Returns: 0.0004
BALANCE
    BTC - {'free': 0.9920622000000004, 'used': 0.0, 'total': 0.9920622000000004}
    ETH - {'free': 0.09999999999999999, 'used': 0.0, 'total': 0.09999999999999999}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.09098681840277778, 'latest_price': 0.087186}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 94 - Timestep: 2018-01-10T22:30:00
---------------------------------------
OHLCV
    O: 0.0863 | C: 0.0849 | V: 10931.8 | T: 2018-01-10T22:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9969 Total Val: 1.0004 P

Placing Order {
    "id": "b8e222a33a8043728259fa5ae320ef54",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.01,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-18T06:03:28.468564",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
Placed order {
    "id": "b8e222a33a8043728259fa5ae320ef54",
    "exchange_id": "paper",
    "exchange_order_id": "4a5ae31f93474684812282d85fabcdbd",
    "price": 0.087186,
    "quantity": 0.01,
    "filled_quantity": 0.01,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-18T06:03:28.468564",
    "opened_time": null,
    "filled_time": "2018-01-18T06:03:28.542661",
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "trades": [],
    "symbol": "ETH/BTC"
}
[{
    "id": "b8e222a33a8043728259fa5ae3

---------------------------------------
Epoch 95 - Timestep: 2018-01-10T23:00:00
---------------------------------------
OHLCV
    O: 0.0848 | C: 0.0841 | V: 7501.0 | T: 2018-01-10T23:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0100
PERFORMANCE
    Cash: 0.9977 Total Val: 1.0003 PnL: 0.0003 Returns: 0.0003
BALANCE
    BTC - {'free': 0.9954909800000004, 'used': 0.0, 'total': 0.9954909800000004}
    ETH - {'free': 0.060000000000000005, 'used': 0.0, 'total': 0.060000000000000005}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.03, 'cost_price': 0.09098681840277778, 'latest_price': 0.084918}
METRICS
    SMA: 5.00
    RSI: 10.00


## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
def live(name, exchange, balance, portfolio, feed, strategy):
    print("WARNING LIVE TRADING!!!!")
    '''
    exp_name = name of your current experiment (multiple runs per strategy)
    '''
    # Where we will save the record
    root = os.path.join(cfg.DATA_DIR, name)

    # This can be retrieved from the user's global config
    store = DATA_STORES[cfg.DATA_STORE](root=root)

    config = {
        'experiment': name, 
        'strategy': strategy.name,
    }
    record = Record(
        config=config,
        portfolio=portfolio,
        balance=balance,
        store=store
    )
    ctx = Context(
        exchange=exchange,
        feed=feed,
        record=record
    )
    feed.initialize()
    
    while True:
        row = feed.next()
        
        if row is not None:
            orders = strategy.process(row, ctx)

            # TODO: Cancelling orders
            # should we auto-cancel any outstanding orders
            # or should we leave this decision up to the Strategy?
            order_manager.cancel_orders(exchange, orders['cancel_ids'])

            # Returns both FILLED and PENDING orders
            # TODO: Order manager handles mapping from Exchange JSON
            # Particularly order types like CLOSED --> FILLED,
            # And OPEN vs PENDING <-- check the 'quantity' vs 'filled' amounts
            orders = order_manager.place_orders(exchange, orders['orders'])
            filled_orders = order_manager.get_filled_orders(orders)

            # Getting the latest prices for each of our positions
            positions = record.portfolio.positions
            latest_prices = get_latest_prices(postions, row)

            # Portfolio needs to know about new filled orders
            record.portfolio.update(filled_orders, latest_prices)

            # Record needs to know about all new orders
            for order in orders:
                record.orders[order.id] = order

            # Update Virtual Balance
            # exchange balance may be impacted by external trading
            for order in filled_orders:
                record.balance.update_by_order(order)

            record.save()
        
        time.sleep(30)
            
    return record

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)